In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
# load MNIST
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)])
dataset = datasets.MNIST(root='./torch-dataset', train=True,
                         download=True, transform=transform)

9913344it [00:00, 12631140.05it/s]                             


Extracting ./torch-dataset\MNIST\raw\train-images-idx3-ubyte.gz to ./torch-dataset\MNIST\raw



29696it [00:00, 14815517.02it/s]         

Extracting ./torch-dataset\MNIST\raw\train-labels-idx1-ubyte.gz to ./torch-dataset\MNIST\raw




1649664it [00:00, 17705880.00it/s]         


Extracting ./torch-dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to ./torch-dataset\MNIST\raw



5120it [00:00, 16506407.75it/s]         

Extracting ./torch-dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./torch-dataset\MNIST\raw

